In [ ]:
# have you installed snn torch?
# %pip install snntorch

In [ ]:
import snntorch as snn


In [ ]:
DATADIR = "/tmp/data"

## Download Dataset using `spikedata` (deprecated)

In [ ]:
# from snntorch.spikevision import spikedata 
# # note that a default transform is already applied to keep things easy

# train_ds = spikedata.DVSGesture("/tmp/data/dvsgesture", train=True, dt=1000, num_steps=500, ds=1)  # ds: spatial compression; dt: temporal compressiondvs_test
# test_ds = spikedata.DVSGesture("/tmp/data/dvsgesture", train=False, dt=1000, num_steps=1800, ds=1)
# test_ds

## Download Dataset using `tonic`

In [ ]:
import tonic
import tonic.transforms as transforms

sensor_size = tonic.datasets.DVSGesture.sensor_size

# Denoise removes isolated, one-off events
# time_window
frame_transform = transforms.Compose([transforms.Denoise(filter_time=10000),
                                      transforms.ToFrame(sensor_size=sensor_size,
                                                         time_window=1000)
                                     ])

train_ds = tonic.datasets.DVSGesture(save_to=DATADIR, transform=frame_transform, train=True)
test_ds = tonic.datasets.DVSGesture(save_to=DATADIR, transform=frame_transform, train=False)

In [ ]:
test_ds

## DataLoaders

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, shuffle=True, batch_size=64)
test_dl = DataLoader(test_ds, shuffle=False, batch_size=64)

In [ ]:
print('the number of items in the dataset is', len(train_dl.dataset))

## Play with Data

In [ ]:
# get a feel for the data
i_item = 42 # random index into a sample
data, label = train_dl.dataset[i_item]
import torch
data = torch.Tensor(data)

print('The data sample has size', data.shape)
print(f"in case you're blind AF, the target is: {label} ({train_ds.classes[label]})")

In [ ]:
train_ds.classes

## Visualize

In [ ]:
import matplotlib.pyplot as plt
import snntorch.spikeplot as splt
from IPython.display import HTML, display
import numpy as np

# flatten on-spikes and off-spikes into one channel
# a = (train_dl.dataset[n][0][:, 0] + train_dl.dataset[n][0][:, 1])
a = (data[:300, 0, :, :] - data[:300, 1, :, :])
# a = np.swapaxes(a, 0, -1)
#  Plot
fig, ax = plt.subplots()
anim = splt.animator(a, fig, ax, interval=200)
HTML(anim.to_html5_video())
# anim.save('nmnist_animation.mp4', writer = 'ffmpeg', fps=50)  

In [ ]:
import matplotlib.pyplot as plt
import torch
plt.imshow(torch.sum(data[:300, 0,:,:], axis=0), cmap='hot')
plt.colorbar()